In [ ]:
#賠償金未繳庫情形
import pandas as pd
import numpy as np
pd.options.display.max_colwidth=0 #顯示全部字串
#付款憑單-集中支付檔-cetralized payment
e106 = pd.read_csv(r'c:\ais\106\資料匯出-20190812095742-付款憑單明細檔.csv')
e107 = pd.read_csv(r'c:\ais\107\資料匯出-20190615134322-付款憑單明細檔.csv')
e108 = pd.read_csv(r'c:\ais\108_01_07\資料匯出-20190827103901-付款憑單明細檔.csv')
e = pd.concat([e106,e107,e108],axis=0)
#請購單-cetralized payment
a106 = pd.read_csv(r'c:\ais\106\資料匯出-20190812095742-請購單明細檔.csv')
a107 = pd.read_csv(r'c:\ais\107\資料匯出-20190615134322-請購單明細檔.csv')
a108 = pd.read_csv(r'c:\ais\108_01_07\資料匯出-20190827103901-請購單明細檔.csv')
a = pd.concat([a106,a107,a108],axis=0)
#收入傳票 income
i106 = pd.read_csv(r'c:\ais\106\資料匯出-20190812095742-收入傳票明細檔.csv',error_bad_lines=False)
i107 = pd.read_csv(r'c:\ais\107\資料匯出-20190615134322-收入傳票明細檔.csv',error_bad_lines=False)
i108 = pd.read_csv(r'c:\ais\108_01_07\資料匯出-20190827103901-收入傳票明細檔.csv',error_bad_lines=False)
i = pd.concat([i106,i107,i108],axis=0)
p106 = pd.read_csv(r'c:\ais\106\資料匯出-20190812095742-明細分類帳.csv')
p107 = pd.read_csv(r'c:\ais\107\資料匯出-20190615134322-明細分類帳.csv')
p108 = pd.read_csv(r'c:\ais\108_01_07\資料匯出-20190827103901-明細分類帳.csv')
p = pd.concat([p106,p107,p108],axis=0)
p['傳票年份']=round(p['傳票日期']/10000,0).astype(int)
e.columns

In [6]:
p[p['總帳科目'].str.contains('2123')
 &p['子目名稱'].str.contains('同濟會捐助款')
 ][['機關名稱', '年度', '傳票類別名稱', '傳票號碼', '總帳科目', '子目名稱', '摘要', '金額']]

,機關名稱,年度,傳票類別名稱,傳票號碼,總帳科目,子目名稱,摘要,金額
23539,花蓮縣光復鄉西富國民小學,106,支出傳票,4,2123,同濟會捐助款,學生營養早餐-張素連墊付,190.0
23540,花蓮縣光復鄉西富國民小學,106,支出傳票,4,2123,同濟會捐助款,105年12月社團活動鐘點費-張國強,7200.0
23541,花蓮縣光復鄉西富國民小學,106,支出傳票,4,2123,同濟會捐助款,105年12月社團活動教師勞保費,14.0
23556,花蓮縣光復鄉西富國民小學,106,支出傳票,7,2123,同濟會捐助款,106年1月課後學習輔導鐘點費-徐悠羚,1040.0
23557,花蓮縣光復鄉西富國民小學,106,支出傳票,7,2123,同濟會捐助款,106年1月課後學習輔導鐘點費-楊威,780.0
23558,花蓮縣光復鄉西富國民小學,106,支出傳票,7,2123,同濟會捐助款,106年1月課後學習輔導鐘點費-吳新明,1040.0
23559,花蓮縣光復鄉西富國民小學,106,支出傳票,7,2123,同濟會捐助款,105年10~12月課後照顧鐘點費-張倪昭葦,6720.0
23560,花蓮縣光復鄉西富國民小學,106,支出傳票,7,2123,同濟會捐助款,105年10~12月課後照顧鐘點費-徐悠羚,4160.0
23561,花蓮縣光復鄉西富國民小學,106,支出傳票,7,2123,同濟會捐助款,105年10~12月課後照顧鐘點費-吳新明,3840.0
23562,花蓮縣光復鄉西富國民小學,106,支出傳票,7,2123,同濟會捐助款,105年10~12月課後照顧鐘點費-劉仁傑,320.0


In [8]:
p[p['摘要'].str.contains('禮品|繳回|結餘|解聘|勞保|賠償',na=False)
 &p['總帳科目'].str.contains('2123')
 &p['子目名稱'].str.contains('捐|募')
 &(~p['子目名稱'].str.contains('教育處|經濟補助|績優社團獎勵|謝師|同濟會',na=False))
 &(~p['摘要'].str.contains('畢業|運動會',na=False))
 ][['機關名稱', '年度', '傳票類別名稱', '傳票號碼', '總帳科目', '子目名稱', '摘要', '金額']]


,機關名稱,年度,傳票類別名稱,傳票號碼,總帳科目,子目名稱,摘要,金額
4848,花蓮縣立吉安國民中學,106,支出傳票,19,2123,應付代收款-教育儲蓄專戶-各界勸募款,支歷年結餘捐贈教育儲蓄戶捐款-702林婉婷--轉存國安郵局個人帳戶,2000.0
12428,花蓮縣花蓮市中正國民小學,106,支出傳票,5,2123,應付代收款-代收代付-校安維護捐款,【轉帳不支】105年12月值日夜人員值勤費公付勞保費,13.0
18455,花蓮縣新城鄉新城國民小學,106,轉帳傳票,4,2123,應收代付款-其他-捐助特教班,科目轉正-結餘款轉入校務發展基金,46.0
18457,花蓮縣新城鄉新城國民小學,106,轉帳傳票,4,2123,應付代收款-午餐專戶-社會捐助午餐費,科目轉正-結餘款轉入校務發展基金,6915.0
18458,花蓮縣新城鄉新城國民小學,106,轉帳傳票,4,2123,應收代付款-其他-民間捐助校務發展基金,科目轉正-各項活動經費結餘款轉入校務發展基金,24235.0
20249,花蓮縣壽豐鄉志學國民小學,106,轉帳傳票,4,2123,應付代收款-其他-捐助學生費,105/12月、106/1月客語教師勞保費,112.0
68910,花蓮縣壽豐鄉豐山國民小學,106,支出傳票,68,2123,應付代收款-其他-各界捐款(捐資興學款),預付106年1月專案教師鐘點公付勞保費(另存入94120帳號),3163.0
68912,花蓮縣壽豐鄉豐山國民小學,106,支出傳票,68,2123,應付代收款-其他-各界捐款(捐資興學款),預付106年1月專案教師鐘點自付勞保費(另存入94120帳號),916.0
68916,花蓮縣壽豐鄉豐山國民小學,106,支出傳票,68,2123,應付代收款-其他-各界捐款(捐資興學款),預付106年2月專案教師鐘點公付勞保費(另存入94120帳號),3081.0
68918,花蓮縣壽豐鄉豐山國民小學,106,支出傳票,68,2123,應付代收款-其他-各界捐款(捐資興學款),預付106年2月專案教師鐘點自付勞保費(另存入94120帳號),916.0


In [ ]:
#捐款支用情形
#106至108年期間應付代收子目期初轉入明細
cb = p[p['總帳科目'].str.contains('2123',na=False)
      &p['摘要'].str.contains('結轉|上期|結帳',na=False)
      ][['機關名稱', '傳票年份', '傳票類別名稱', '傳票號碼', '總帳科目', '子目名稱',  '摘要', '金額'
      ]].sort_values(['機關名稱'],ascending=[False])
cb = cb.rename({'金額': '期初轉入數'}, axis='columns')
cb3y = pd.DataFrame(pd.pivot_table(cb
                                  ,index=['機關名稱','子目名稱']
                                  ,columns=['傳票年份'],values=['期初轉入數']
                                  ,aggfunc=np.sum
                                  ,fill_value=0
                                  ).astype('int').to_records()
                  )
#106至108年期間應付代收子目支用明細
ce = p[p['總帳科目'].str.contains('2123',na=False)
      &p['借貸方註記']==1
      ][['機關名稱', '傳票年份', '傳票類別名稱', '傳票號碼', '總帳科目', '子目名稱',  '摘要', '金額'
        ]].sort_values(['機關名稱'],ascending=[False])
ce = ce.rename({'金額': '支用數'}, axis='columns')
ce3y = pd.DataFrame(pd.pivot_table(ce
                                  ,index=['機關名稱','子目名稱']
                                  ,columns=['傳票年份'],values=['支用數']
                                  ,aggfunc=np.sum
                                  ,fill_value=0
                                  ).astype('int').to_records()
                   )
# 106至108年應付代收科目支用率未達1成明細
c3y = pd.merge(cb3y, ce3y, how='outer', on=['機關名稱','子目名稱'])
c3y["('支用數', 106)"].fillna(0, inplace=True)
c3y["('支用數', 107)"].fillna(0, inplace=True)
c3y["('支用數', 108)"].fillna(0, inplace=True)
c3y["106支用率"] = c3y["('支用數', 106)"]/c3y["('期初轉入數', 106)"]
c3y["107支用率"] = c3y["('支用數', 107)"]/c3y["('期初轉入數', 107)"]
c3y["108支用率"] = c3y["('支用數', 108)"]/c3y["('期初轉入數', 108)"]
c3y["106支用率"].fillna(0, inplace=True)
c3y["107支用率"].fillna(0, inplace=True)
c3y["108支用率"].fillna(0, inplace=True)

c3y[(c3y['子目名稱'].str.contains("非政府機關委託|非政府機關補助|助學金|發展教育基金|教育發展|獎學金|獎助金")
    |c3y['子目名稱'].str.contains("畢業|教育儲蓄專戶|富邦基金會學童認養金|慈善|贊助|民間團體委託|捐贈|捐資興學|捐助|同濟會|急難慰問金")
    |c3y['子目名稱'].str.contains("玉山文教基金會|仁愛|指定用途")      
    )
   & (~c3y["機關名稱"].str.contains("花蓮縣政府教育處"))
   & (c3y["('期初轉入數', 108)"] > 0)
   & (c3y["108支用率"] < 0.1)
   & (c3y["107支用率"] < 0.1)
   & (c3y["106支用率"] < 0.1)
   ]#.to_csv("low_used_donations.csv")

In [ ]:
#前2年核有期初轉入數惟未有支用數之代收代付或代辦、保管款科目明細
c3y[(c3y["('期初轉入數', 108)"] > 0)
   &(c3y["('期初轉入數', 107)"] > 0)
   &(c3y["('支用數', 108)"]  == 0)
   &(c3y["('支用數', 107)"]  == 0)
   &(~c3y['子目名稱'].str.contains("非政府機關委託|非政府機關補助|助學金|發展教育基金|教育發展|獎學金|獎助金")
    &~c3y['子目名稱'].str.contains("畢業|教育儲蓄專戶|富邦基金會學童認養金|慈善|贊助|民間團體委託|捐贈|捐資興學|捐助|同濟會|急難慰問金")
    &~c3y['子目名稱'].str.contains("玉山文教基金會|仁愛基金|指定用途")
    &~c3y['子目名稱'].str.contains("代扣|勞保|勞退|退休|健保|退撫|所得稅|公保|預付|代扣款|補充保費|值勤費")         
    )].to_csv("unused_collections.csv")

In [ ]:
#各校賠償款收入繳庫情形
#各校代收賠償款專戶期初餘額情形
#damage incomne collections
db = p[p['子目名稱'].str.contains('賠償|損害',na=False)
      &p['摘要'].str.contains('結轉|上期|結帳',na=False)
      &(p['傳票年份'] == 106)
      ][['機關名稱', '傳票年份', '傳票類別名稱', '傳票號碼', '總帳科目', '子目名稱',  '摘要', '金額'
       ]].sort_values(['機關名稱', '傳票年份', '傳票號碼'],ascending=[False, True, True])
db = db.rename({'金額': '期初轉入數','傳票年份':'年度'}, axis='columns')
db.to_csv("damage_106_balance.csv")

db3y = pd.DataFrame(pd.pivot_table(db
                                  ,index=['機關名稱']
                                  ,values=['期初轉入數']
                                  ,aggfunc=np.sum 
                                  ,fill_value=0
                                  ).astype('int').to_records())

#各校代收賠償款專戶收入情形
#damage incomne collections
di = i[(i['摘要'].str.contains('賠|損|毀|公物',na=False)
       |i['子目名稱'].str.contains('賠|損|毀|公物',na=False)
       )
       &~i['子目名稱'].str.contains('保證|補助|強制',na=False)
       &~i['摘要'].str.contains('督導費|工程|腸胃不適|學生醫療',na=False)
       &~i['貸方科目代碼'].str.contains('4Y',na=False)
      ][['機關名稱','年度', '傳票號碼','貸方科目代碼', '子目名稱',  '摘要', '貸方金額',
      ]].sort_values(['機關名稱', '年度', '傳票號碼'],ascending=[False, True, True])
di = di.rename({'貸方金額':'收入數'},axis='columns')
di['傳票類別名稱']='收入傳票'
di.to_csv("damage_income.csv")
di3y = pd.DataFrame(pd.pivot_table(di
                                  ,index=['機關名稱']
                                  ,values=['收入數']
                                  ,aggfunc=np.sum 
                                  ,fill_value=0
                                  ).astype('int').to_records())
#各校代收賠償款專戶支出情形
de = p[(p['摘要'].str.contains('賠|公物',na=False)
       |p['子目名稱'].str.contains('賠|毀|公物',na=False)
       )
      &~p['子目名稱'].str.contains('保證|補助',na=False)
      &~p['摘要'].str.contains('更正|轉正|督導費|工程|腸胃不適|學生醫療|研習|給付差額',na=False) 
      &p['總帳科目'].str.contains('2123',na=False)
      &p['借貸方註記']==1
      ][['機關名稱', '傳票年份', '傳票類別名稱', '傳票號碼', '總帳科目', '子目名稱',  '摘要', '金額'
       ]].sort_values(['機關名稱', '傳票年份', '傳票號碼'],ascending=[False, True, True])
de = de.rename({'金額': '支用數','傳票年份':'年度'}, axis='columns')
de.to_csv("damage_expend.csv")
de3y = pd.DataFrame(pd.pivot_table(de
                                  ,index=['機關名稱']
                                  ,values=['支用數']
                                  ,aggfunc=np.sum 
                                  ,fill_value=0
                                  ).astype('int').to_records())

#各校代收賠償款專戶繳庫情形
dt = p[(p['摘要'].str.contains('賠|公物',na=False)
       |p['子目名稱'].str.contains('賠|毀|公物',na=False)
       )
      &~p['子目名稱'].str.contains('保證|補助',na=False)
      &~p['摘要'].str.contains('更正|轉正|督導費|工程|腸胃不適|學生醫療|研習|給付差額',na=False) 
      &p['總帳科目'].str.contains('2123',na=False)
      &p['摘要'].str.contains('庫',na=False) 
      &p['借貸方註記']==1
      ][['機關名稱', '傳票年份', '傳票類別名稱', '傳票號碼', '總帳科目', '子目名稱',  '摘要', '金額'
       ]].sort_values(['機關名稱', '傳票年份', '傳票號碼'],ascending=[False, True, True])
dt = dt.rename({'金額': '繳庫數','傳票年份':'年度'}, axis='columns')
dt.to_csv("damage_treasuary.csv")
dt3y = pd.DataFrame(pd.pivot_table(dt
                                  ,index=['機關名稱']
                                  ,values=['繳庫數']
                                  ,aggfunc=np.sum 
                                  ,fill_value=0
                                  ).astype('int').to_records())

d3y = pd.merge(di3y, de3y, how='outer', on=['機關名稱'])
d3y = pd.merge(d3y,  dt3y, how='outer', on=['機關名稱'])
d3y = pd.merge(d3y,  db3y, how='outer', on=['機關名稱'])
d3y = d3y.fillna(0)
d3y['代收總數']=d3y['期初轉入數']+d3y['收入數']
tot = d3y.sum(numeric_only=True)
tot['機關名稱']='合計'
tot = pd.DataFrame(tot)
tot = tot.T
d3y = pd.concat([tot, d3y]).reset_index()
#d3y = d3y.fillna('合計')
d3y['支用率']=d3y['支用數']/d3y['代收總數']
d3y['繳庫率']=d3y['繳庫數']/d3y['代收總數']
d3y =d3y[['機關名稱', '期初轉入數', '收入數', '代收總數', '支用數', '支用率', '繳庫數', '繳庫率']]
d3y.to_csv("damage_collections.csv")
d3y

In [ ]:
# 106至108年捐款科目支用比率未達1成明細
po13y = pd.merge(po3y, p13y, how='left', left_on=['機關名稱','子目名稱'], right_on = ['機關名稱','子目名稱'])
po13y["('支出額', 106)"].fillna(0, inplace=True)
po13y["('支出額', 107)"].fillna(0, inplace=True)
po13y["('支出額', 108)"].fillna(0, inplace=True)
po13y["106支用比率"] = po13y["('支出額', 106)"]/po13y["('期初餘額', 106)"]
po13y["107支用比率"] = po13y["('支出額', 107)"]/po13y["('期初餘額', 107)"]
po13y["108支用比率"] = po13y["('支出額', 108)"]/po13y["('期初餘額', 108)"]
po13y["106支用比率"].fillna(0, inplace=True)
po13y["107支用比率"].fillna(0, inplace=True)
po13y["108支用比率"].fillna(0, inplace=True)
po13y
po13y[(po3y['子目名稱'].str.contains("非政府機關委託|非政府機關補助|助學金|發展教育基金|教育發展|獎學金|獎助金")
       |po3y['子目名稱'].str.contains("畢業|教育儲蓄專戶|富邦基金會學童認養金|慈善|贊助|民間團體委託|捐贈|捐資興學|捐助|同濟會|急難慰問金")
       |po3y['子目名稱'].str.contains("玉山文教基金會|仁愛|指定用途")      
       )
     & (po13y["('期初餘額', 108)"] > 0)
     & (po13y["108支用比率"] < 0.1)
     & (po13y["107支用比率"] < 0.1)
     & (po13y["106支用比率"] < 0.1)
     ].to_csv("low_used_donations.csv")


In [ ]:
# 前2年核有期初餘額惟未有支用數之代收代付或代辦、保管款科目明細
po13y = pd.merge(po3y, p13y, how='left', left_on=['機關名稱','子目名稱'], right_on = ['機關名稱','子目名稱'])
po13y[["('支出額', 106)","('支出額', 107)","('支出額', 108)"]].fillna(0, inplace=True)
po13y[  (po13y["('期初餘額', 108)"] > 0)
      & (po13y["('期初餘額', 107)"] > 0)
      & (po13y["('支出額', 108)"]  == 0)
      & (po13y["('支出額', 107)"]  == 0)
      & (~po3y['子目名稱'].str.contains("非政府機關委託|非政府機關補助|助學金|發展教育基金|教育發展|獎學金|獎助金")
        &~po3y['子目名稱'].str.contains("畢業|教育儲蓄專戶|富邦基金會學童認養金|慈善|贊助|民間團體委託|捐贈|捐資興學|捐助|同濟會|急難慰問金")
        &~po3y['子目名稱'].str.contains("玉山文教基金會|仁愛基金|指定用途")
        &~po3y['子目名稱'].str.contains("代扣|勞保|勞退|退休|健保|退撫|所得稅|公保|預付|代扣款|補充保費|值勤費")         
        )
     ]

In [ ]:
#106至108年期間應付代收子目代辦經費支出紀錄
p1 = p[p['子目名稱'].str.contains('應付代收',na=False)
      &p['借貸方註記']==1
      ][['機關名稱','總帳科目', '子目代碼', '子目名稱', '傳票年份', '摘要', '借貸方註記', '金額',
         '來源傳票號碼', '經費來源', '經費來源名稱'
        ]
       ].sort_values(['機關名稱'],ascending=[False])
p1 = p1.rename({'金額': '支出額'}, axis='columns')
p13y = pd.DataFrame(pd.pivot_table(p1
                                  ,index=['機關名稱','子目名稱']
                                  ,columns=['傳票年份'],values=['支出額']
                                  ,aggfunc=np.sum
                                  ,fill_value=0
                                  ).astype('int').to_records()
                   )

In [ ]:
#106至108年期間應付代收子目代辦經費期初金額3年均大於1000元者、排除捐款、保管款、代收代付、代扣款、學生代收代辦費用、午餐、基金來源
#列出各年份應付代收子目期初金額
po = p[p['子目名稱'].str.contains('應付代收',na=False)
 &p['摘要'].str.contains('結轉|上期|結帳',na=False)
 ][['機關名稱','總帳科目', '子目代碼', '子目名稱', '傳票年份', '摘要', '借貸方註記', '金額',
   '來源傳票號碼', '經費來源', '經費來源名稱'
   ]
].sort_values(['機關名稱'],ascending=[False])
po = po.rename({'金額': '期初餘額'}, axis='columns')
po3y = pd.DataFrame(pd.pivot_table(po
                                 ,index=['機關名稱','子目名稱']
                                 ,columns=['傳票年份'],values=['期初餘額']
                                 ,aggr_func=np.sum
                                 ,fill_value=0
                                 ).astype('int').to_records()
                  )
po3y[(~po3y['子目名稱'].str.contains("非政府機關委託|非政府機關補助|助學金|發展教育基金|教育發展|獎學金|獎助金")
    &~po3y['子目名稱'].str.contains("畢業|教育儲蓄專戶|富邦基金會學童認養金|慈善|贊助|民間團體委託|捐贈|捐資興學|捐助|同濟會|急難慰問金")
    &~po3y['子目名稱'].str.contains("玉山文教基金會")
    &~po3y['子目名稱'].str.contains("學生繳費|幼兒園|幼稚園|簿本費|教科書")     
    &~po3y['子目名稱'].str.contains("午餐") 
    &~po3y['子目名稱'].str.contains("合作社|資源回收|場地設施使用費|租金收入|利息收入|收回薪津") #基金來源
    &~po3y['子目名稱'].str.contains("押金|保固金|保證金")
    &~po3y['子目名稱'].str.contains("代扣|勞保|勞退|退休|健保|退撫|所得稅|公保|預付|代扣款|補充保費|值勤費")
    &~po3y['子目名稱'].str.contains("薪資暫存")
    )
   &(po3y["('期初餘額', 106)"]>1000)
   &(po3y["('期初餘額', 108)"]>1000)
   &(po3y["('期初餘額', 107)"]>1000)
   ].sort_values(['機關名稱', '子目名稱', "('期初餘額', 108)"], ascending=[False, False, False])

In [ ]:
p[p['子目名稱'].str.contains('應付代收款－代收代付－其他',na=False)
 &p['機關名稱'].str.contains('光復國民小學',na=False)
 ][['機關名稱','總帳科目', '子目代碼', '子目名稱', '傳票年份', '摘要', '借貸方註記', '金額',
    '來源傳票號碼', '經費來源', '經費來源名稱'
   ]]

In [ ]:
#應付代收款項下校務基金用途明細
# general donation
gd = p[((p['子目名稱'].str.contains('校務發展|校務基金|校務推動|校務推動費|校務經費',na=False))
       |(p['摘要'].str.contains('校務發展|校務基金|校務推動|校務推動費|校務經費',na=False)))
      &~(p['摘要'].str.contains('上期結轉',na=False))
      &(p['借貸方註記']==1)
      &(p['總帳科目'].str.contains('2123'))
      ]
gd[['機關名稱', '傳票日期', '傳票類別名稱', '傳票號碼', '總帳科目', '子目名稱', "摘要",'借貸方註記','金額']].to_csv("donation_for_general_usage.csv")

In [ ]:
#應付代收款項下校務基金來源明細
# general donation
gd = p[((p['子目名稱'].str.contains('校務發展|校務基金|校務推動|校務推動費|校務經費',na=False))
       |(p['摘要'].str.contains('校務發展|校務基金|校務推動|校務推動費|校務經費',na=False)))
      &~(p['摘要'].str.contains('上期結轉',na=False))
      &(p['借貸方註記']==2)
      &(p['總帳科目'].str.contains('2123'))
      ]
gd[['機關名稱', '傳票日期', '傳票類別名稱', '傳票號碼', '總帳科目', '子目名稱', "摘要",'借貸方註記','金額']].to_csv("general_donation_source.csv")

In [ ]:
#代收代付校務發展基金機關收支金額
pd.pivot_table(gd
              ,index=['機關名稱','子目名稱']
              ,columns=['借貸方註記']
              ,values=['金額']
              ,fill_value=0
              ).astype('int')

In [ ]:
#列出各年份應付代收子目期初金額
p[p['子目名稱'].str.contains('應付代收',na=False)
 &p['摘要'].str.contains('結轉|上期|結帳',na=False)
 ][['機關名稱','總帳科目', '子目代碼', '子目名稱', '傳票年份', '摘要', '借貸方註記', '金額',
   '來源傳票號碼', '經費來源', '經費來源名稱'
   ]
].sort_values(['機關名稱'],ascending=[False])
p3y = pd.DataFrame(pd.pivot_table(p
                                 ,index=['機關名稱','子目名稱']
                                 ,columns=['傳票年份'],values=['金額'],fill_value=0
                                 ).astype('int').to_records()
                  )

In [ ]:
#106至108年期間應付代收子目期初金額逐年遞增者、排除捐款、補助
p3y[(~p3y['子目名稱'].str.contains("非政府機關委託補助|足球隊補助費|獎助學金|捐款|教職員工午餐費|拾金不昧|法院執行|幼兒午餐|值日夜經費|教師事病假代課費|值勤薪資|幼兒園-課後留園|幼兒園學生平安保險費|幼兒園學生餐點費|短代外聘教師年終獎金|導師費|優惠存款|臨時人員薪資|勞工退休|法院強制命令|代收代付-所得|代課鐘點費|加班費|兒午餐費|幼兒園材料費|幼兒園-保險費|幼兒園平安保險費|幼兒園-材料費|幼兒園午餐費|幼兒園-點心午餐費|健保|勞保|公保|退輔基金|休假補助款|所得稅|退撫基金|補充保費|薪資所得|勞退|退休準備|活動費|保費|點心費"))
   &(p3y["('金額', 106)"]>0)
   &(p3y["('金額', 108)"]>p3y["('金額', 107)"])
   &(p3y["('金額', 107)"]>p3y["('金額', 106)"])
   ].sort_values(['子目名稱', '機關名稱', "('金額', 108)"], ascending=[False, False, False])

In [ ]:
r[r['摘要'].str.contains('賠償收入繳庫',na=False)
 ][['機關名稱','傳票號碼', '傳票明細序號', '傳票類別代碼', '傳票類別名稱', '子目代碼', '子目名稱'
  ,'分支計畫代碼', '分支計畫名稱', '用途別代碼', '預計轉入科目代碼', '預計轉入子目代碼', '預計轉入子目名稱', '委受名稱'
  ,'摘要', '貸方金額', '原始憑證種類', '原始憑證號數', '貸方科目代碼', '經費來源', '經費來源名稱']
  ]

In [ ]:
ap[(ap['請購金額']==85289)]

In [ ]:
ap[(ap['機關名稱'].str.contains(r"澔學",na=False))]

In [ ]:
#服裝費支付員工活動服裝
ap[(ap['用途代碼'].str.contains('325'))
  &(ap['摘要'].str.contains("服裝"))        
  ][['機關名稱','總帳科目代碼', '子目名稱', '分支計畫名稱','用途代碼', '摘要', '請購金額']]

In [ ]:
#以公款支付罰款
cp[cp['摘要'].str.contains('罰|滯|違|未足額進用')][['摘要']]

In [ ]:
#是否以非公關費支付餐敘、禮品
cp[(cp['摘要'].str.contains(r"招待|禮|餐"))
   &(~cp['摘要'].str.contains('學校午餐|貧困午餐|稽查|接力|賽|國手|白禮袋|廚工|代扣|設備|餐飲科|明禮|餐廳|慈輝|教室|誤餐|夜輔|廚房|典禮|學生|午餐秘書|姜禮誠|日誌'))
   &(~cp['用途別代嗎'].str.contains('291|27F'))
   ][['機關名稱','借方科目碼', '分支計畫名稱','用途別代嗎','摘要', '實付數', '預計轉入科目代碼']]

In [ ]:
#是否以非公關費支付餐敘、禮品
ap1 = ap[(~ap['用途代碼'].str.contains('291|27F'))
        &(ap['摘要'].str.contains("招待|禮"))        
        ]
ap2[(~ap2['摘要'].str.contains('學校午餐|書法營|畢業|門鎖|試辦計畫|白禮袋|家長|委員|期末|貧困午餐|兒童節|幼兒園|園遊會|稽查|活動|木工班|明禮|禮班|禮堂|接力|賽|國手|廚工|代扣|設備|體育班|義工|志工|餐飲科|餐廳|慈輝|教室|誤餐|夜輔|廚房|典禮|學生|午餐秘書|姜禮誠|日誌'))
  ][['機關名稱','總帳科目代碼', '子目名稱', '分支計畫名稱','用途代碼', '摘要', '請購金額']]

In [ ]:
#工程管理費超支
p1 = p[p['摘要'].str.contains('花崗國中Q棟',na=False)]
p1[~p1['摘要'].str.contains('保證')
  ][['機關名稱', '年度'
    , '教育部機關代碼', '傳票類別名稱', '傳票號碼', '序號', '借貸方註記', '傳票日期', '金額'
    ,'摘要', '總帳科目', '用途別代碼', '分支計畫代碼', '分支計畫名稱', '子目代碼', '子目名稱'
    , '過帳日期', '經費來源', '經費來源名稱']]